In [1]:
import csv
import cv2
import numpy as np

lines = []
with open('Data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

print(lines[0])

from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(lines, test_size=0.2)

['C:\\Users\\y\\Desktop\\TrainingData\\IMG\\center_2018_01_09_03_33_02_577.jpg', 'C:\\Users\\y\\Desktop\\TrainingData\\IMG\\left_2018_01_09_03_33_02_577.jpg', 'C:\\Users\\y\\Desktop\\TrainingData\\IMG\\right_2018_01_09_03_33_02_577.jpg', '0', '0', '0', '4.364454E-06']


In [3]:
import sklearn
import os
def generator(samples, batch_size=100):
    num_samples = len(samples)
    while 1: 
        
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = 'Data/IMG/'+batch_sample[0].split('\\')[-1]
                center_image = cv2.imread(name)
                center_angle = float(batch_sample[3])
                namel = 'Data/IMG/'+batch_sample[1].split('\\')[-1]
                l_image = cv2.imread(namel)
                l_angle = float(batch_sample[3]) + 0.15                
                namer = 'Data/IMG/'+batch_sample[2].split('\\')[-1]
                r_image = cv2.imread(namer)
                r_angle = float(batch_sample[3]) - 0.15                             
                
                images.append(center_image)
                images.append(l_image)
                images.append(r_image)
                images.append(cv2.flip(center_image,1))
                images.append(cv2.flip(l_image,1))
                images.append(cv2.flip(r_image,1))
                angles.append(center_angle)
                angles.append(l_angle)
                angles.append(r_angle)
                angles.append(center_angle*-1.0)
                angles.append(l_angle*-1.0)
                angles.append(r_angle*-1.0)
            
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

In [4]:

train_generator = generator(train_samples, batch_size=100)
validation_generator = generator(validation_samples, batch_size=100)

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Activation, Cropping2D
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
model.add(Lambda(lambda x:x/255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25), (0,0))))
model.add(Convolution2D(24, 5, 5, subsample =(2,2)))
model.add(Activation('relu'))
model.add(Convolution2D(36, 5, 5, subsample =(2,2)))
model.add(Activation('relu'))
model.add(Convolution2D(48, 5, 5, subsample =(2,2)))
model.add(Activation('relu'))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit_generator(train_generator, samples_per_epoch=6*len(train_samples), validation_data=validation_generator, nb_val_samples=6*len(validation_samples), nb_epoch=3)
model.save('model.h5')

Using TensorFlow backend.


Epoch 1/3
42438/42438 [==============================] - 96s - loss: 0.0209 - val_loss: 0.0185
Epoch 2/3
42438/42438 [==============================] - 77s - loss: 0.0172 - val_loss: 0.0170
Epoch 3/3
42438/42438 [==============================] - 76s - loss: 0.0161 - val_loss: 0.0165
